In [1]:
import re
import toml
import json
import numpy as np
from scipy.constants import R
from moleculegraph.molecule_utils import sort_force_fields, sort_graph_key
from moleculegraph.general_utils import read_pair_potentials, read_bond_potentials, read_angle_potentials, read_torsion_potentials

# Define a function to replace 'xx' with a given number
def replace_xx_with_number(input_string, number):
    return input_string.replace("xx", str(number))

def get_xx_combinations( string, pattern = r'\[CHxx_alkane\]' ):
    combinations = []

    matches = re.findall(pattern, string)

    def generate_combinations(current_string, index):
        # If index reaches the end, append the current string to combinations
        if index == len(matches):
            combinations.append(current_string)
            return
        
        # Replace 'xx' with 1, 2, and 3 and proceed with next index
        for j in range(1, 4):
            if j == 1:
                j = ""
            new_string = current_string.replace(matches[index], replace_xx_with_number(matches[index], j), 1)
            generate_combinations(new_string, index + 1)

    # Generate combinations
    generate_combinations(string, 0)

    return combinations

# Write force field toml files #

This notebook is used to translate the TAMie force field files into a moleculegraph compatible toml file. The units are also converted to LAMMPS standard units (kcal/mol).

### Note:
In TOML one cannot mix integers and floats in one list! The nharmonic style needs as first parameter the length of the expansion, hence an integer.
Therefore, it is better to save the force field for alcohols as json file. pyLAMMPS can read both json or toml.

In [2]:
# Define the TAMie force field folder
path_to_tamie    = "force-fields/UA_TAMie_thiols/"

# Define the output toml file
output_file_name = "force-fields/ff_UA_TAMie_thiols.toml"

# Read in the force field files using moleculegraph

pair_pots    = read_pair_potentials( path_to_tamie + "pair_potentials" )
bond_pots    = read_bond_potentials( path_to_tamie + "bond_potentials" )  
angle_pots   = read_angle_potentials( path_to_tamie + "angle_potentials" )
torsion_pots = read_torsion_potentials( path_to_tamie + "torsion_potentials" )

# Converstion factor from unit Kelvin to kcal/mol (K * J/(K*mol) * kcal/J)
KB_kcal = R / 4184

# Define pair interactions (sigma in Angstrom, epsilon in kcal/mol)
for p in pair_pots:
    pair_pots[p]["vdw_style"] = "mie/cut"
    pair_pots[p]["coulomb_style"] = "" if pair_pots[p]["charge"] == 0.0 else "coul/long" 
    pair_pots[p]["epsilon"] = round( pair_pots[p]["epsilon"] * KB_kcal, 4 )
    pair_pots[p]["n"] = pair_pots[p]["m"]
    pair_pots[p]["m"] = 6

    if "H" in p and not p[0].lower() == "c":
        H_count = p.count('H')
        print(f"United atom found with central atom other than carbon: '{p}'. Reduce the molecular mass by the number of added hydrogens: '{H_count}'!\n")
        pair_pots[p]["mass"] -= 1.008 * H_count

# Define bonded interactions (Spring constant in kcal/mol / Angstrom^2. As TAMie utilize fixed bond, use empirical value: https://doi.org/10.1021/acs.jced.9b01031 )
additional_bonds = {  }
for p in bond_pots:
    bond_pots[p]["style"] = "harmonic"
    bond_pots[p]["p"] = [ 200.0, bond_pots[p]["p"][0] ]

    # TAMie provides generic xx veresion. Write them out to get all combinations
    if "CHxx" in p:
        print(f"Bond found that has generic 'CHxx' in key: '{p}'! Add unique combinations from CH to CH3!\n")
        for comp in np.unique( [ sort_graph_key(key)  for key in get_xx_combinations( p ) ] ):
            additional_bonds[comp] = { **bond_pots[p] , "list": sort_force_fields( comp[1:-1].split("][") ).tolist()   }

# Update with the additional ones
bond_pots.update( additional_bonds )

# Define angle interactions (Spring constant in kcal/mol / rad^2) --> introduce the factor 1/2 in the constant (see LAMMPS documentary)
additional_angle = {  }
for p in angle_pots:
    angle_pots[p]["style"] = "harmonic"
    angle_pots[p]["p"] = [ round(angle_pots[p]["p"][1] * KB_kcal/2, 4 ), angle_pots[p]["p"][0] ]

    
    # TAMie provides generic xx veresion. Write them out to get all combinations
    if "CHxx" in p:
        print(f"Angle found that has generic 'CHxx' in key: '{p}'! Add unique combinations from CH to CH3!\n")
        for comp in np.unique( [ sort_graph_key(key)  for key in get_xx_combinations( p ) ] ):
            additional_angle[comp] = { **angle_pots[p] , "list": sort_force_fields( comp[1:-1].split("][") ).tolist()  }

# Update with the additional ones
angle_pots.update( additional_angle )

# Define dihedral interactions (Spring constant in kcal/mol)
# Leave out last value from the torsion potdentials, as its always 0.0.
additional_torsion = {}
for p in torsion_pots:
    if torsion_pots[p]["type"] == 1:
        torsion_pots[p]["style"]     = "opls"
        # LAMMPS opls uses 1/2*K, but TAMie uses K, hence multiply with two.
        # TAMie utilizes a C0 constant and end at C3 while LAMMPS starts with C1 and ends with C4, hence leave out first entry and add an 0.0 for C4
        torsion_pots[p]["p"]         = [ round( x * KB_kcal * 2, 4 ) for x in torsion_pots[p]["p"][1:-1] ] + [ 0.0 ]

    elif torsion_pots[p]["type"] == 6:
        torsion_pots[p]["style"]     = "nharmonic"
        # No prefactor is used. Add number of parameters of the nharmonic 
        torsion_pots[p]["p"]         = [ len(torsion_pots[p]["p"][:-1]) ] + [ round( x * KB_kcal, 4 ) for x in torsion_pots[p]["p"][:-1] ]
    
    # TAMie provides generic xx veresion. Write them out to get all combinations
    if "CHxx" in p:
        print(f"Torsion found that has generic 'CHxx' in key: '{p}'! Add unique combinations from CH to CH3!\n")
        for comp in np.unique( [ sort_graph_key(key)  for key in get_xx_combinations( p ) ] ):
            additional_torsion[comp] = { **torsion_pots[p] , "list": sort_force_fields( comp[1:-1].split("][") ).tolist() }

# Update with the additional ones
torsion_pots.update( additional_torsion )

# Write toml file
forcefield = {}
forcefield["atoms"]    = pair_pots
forcefield["bonds"]    = bond_pots
forcefield["angles"]   = angle_pots
forcefield["torsions"] = torsion_pots



with open(output_file_name, "w") as toml_file:
    toml.dump(forcefield, toml_file)


with open(output_file_name.replace("toml","json"), "w") as toml_file:
    json.dump(forcefield,toml_file,indent=1)

United atom found with central atom other than carbon: 'SH_thiol'. Reduce the molecular mass by the number of added hydrogens: '1'!

Angle found that has generic 'CHxx' in key: '[CHxx_alkane][CH2_alkane][CHxx_alkane]'! Add unique combinations from CH to CH3!

Angle found that has generic 'CHxx' in key: '[CH2_thiol][CH2_alkane][CHxx_alkane]'! Add unique combinations from CH to CH3!

Torsion found that has generic 'CHxx' in key: '[CHxx_alkane][CH2_alkane][CH2_alkane][CHxx_alkane]'! Add unique combinations from CH to CH3!

Torsion found that has generic 'CHxx' in key: '[CHxx_alkane][CH2_alkane][CH2_thiol][SH_thiol]'! Add unique combinations from CH to CH3!

Torsion found that has generic 'CHxx' in key: '[CH2_thiol][CH2_alkane][CH2_alkane][CHxx_alkane]'! Add unique combinations from CH to CH3!

